In [32]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import random

In [33]:
# Function to extract Product Title
def get_title(soup):

    try:
        title = soup.find("span", attrs={"class":'VU-ZEz'}).text.strip()

    except AttributeError:
        title = np.nan

    return title


In [34]:
# Function to extract Product Price (current after discount)
def get_Currentprice(soup):

    try:
        price = soup.find("div", attrs={"class":'Nx9bqj CxhGGd'}).text
        
    except AttributeError:
        price = np.nan

    return price

In [35]:
# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("div", attrs={"class":'XQDdHH'}).text
    except AttributeError:
        rating = np.nan	

    return rating

In [36]:
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
         
        review_count = soup.find("span", attrs={"class": "Wphh3N"})
        if review_count:
            text = review_count.get_text(" ", strip=True)
            ratings = text.split("&")[0].strip() 

    except AttributeError:
        review_count = np.nan	

    return ratings

In [37]:
# Function to extract Product Price (original before discount)

def get_Originalprice(soup):

    try:
        price = soup.find("div", attrs={"class":'yRaY8j A6+E6v'}).text
        
    except AttributeError:
        price = np.nan

    return price

In [38]:
# Function to extract Total off percentage

def get_OffPercentage(soup):
    try:
        offpercentage = soup.find("div", attrs={"class":'UkUFwK WW8yVX'}).text
    
    except AttributeError:
        offpercentage = np.nan
    
    return offpercentage

In [ ]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
    
    links_list = []

    for page in range(1, 6):

        URL = f"https://www.flipkart.com/search?q=laptop+under+80000&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={page}"
        webpage = requests.get(URL, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "html.parser")

        links = soup.find_all("a", attrs={'class':'CGtC98'})

        for link in links:
            href = link.get('href')
            if href and href.startswith('/'):
                links_list.append(href)
                if len(links_list) >= 100:
                    break
        
        time.sleep(random.uniform(2, 5))

        if len(links_list) >= 100:
            break

    d = {"title":[], "current price":[], "original price":[], "rating":[], "reviews":[], "off percentage":[]}

    # Loop for extracting product details from each link 
    for link in links_list:
        product = "https://www.flipkart.com" + link
        new_webpage = requests.get(product, headers=HEADERS)
        time.sleep(random.uniform(1, 4))
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['current price'].append(get_Currentprice(new_soup))
        d['original price'].append(get_Originalprice(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['off percentage'].append(get_OffPercentage(new_soup))

    
    flipkart_df = pd.DataFrame.from_dict(d)
    print(flipkart_df.isna().sum())
    flipkart_df = flipkart_df.dropna()

title             0
current price     0
original price    0
rating            0
reviews           0
off percentage    0
dtype: int64


In [ ]:
# To remove the Ruppee symbol
flipkart_df['original price'] = flipkart_df['original price'].str.replace('₹', '', regex=False).str.replace(',', '', regex=False).astype(float)
flipkart_df['current price'] = flipkart_df['current price'].str.replace('₹', '', regex=False).str.replace(',', '', regex=False).astype(float)
flipkart_df.to_csv("flipkart_data.csv", header=True, index=False)

flipkart_df
# len(links_list)

,title,current price,original price,rating,reviews,off percentage
0,MOTOROLA Motobook 60 Full Metal OLED (i5 14th ...,49999.0,93690.0,4.4,"1,365 Ratings",46% off
1,MOTOROLA Motobook 60 Full Metal OLED (i5 14th ...,49999.0,93690.0,4.4,"1,365 Ratings",46% off
2,Samsung Galaxy Book5 AI Metal Intel Core Ultra...,77990.0,99990.0,4.5,261 Ratings,22% off
3,"ASUS Vivobook 16X For Creator, Intel H Series ...",64990.0,98990.0,4.2,984 Ratings,34% off
4,Lenovo IdeaPad Slim 5 WUXGA OLED Full Metal Bo...,64990.0,97290.0,4.7,20 Ratings,33% off
...,...,...,...,...,...,...
95,"ASUS Vivobook 16X For Creator, Intel H-Series ...",79965.0,92999.0,4.2,681 Ratings,14% off
96,Lenovo 100e Chromebook Gen 4 MediaTek Kompanio...,8499.0,16298.0,3.9,"1,357 Ratings",47% off
97,Acer Aspire 3 Intel Celeron Dual Core - (8 GB/...,15990.0,33399.0,3.8,"5,717 Ratings",52% off
98,HP Victus Intel Core i7 12th Gen 12650H - (16 ...,72990.0,111500.0,3.3,21 Ratings,34% off
